<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/20220114_hj_know_2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

     |████████████████████████████████| 308 kB 20.6 MB/s 
     |████████████████████████████████| 209 kB 53.2 MB/s 
     |████████████████████████████████| 80 kB 9.3 MB/s 
     |████████████████████████████████| 75 kB 4.8 MB/s 
     |████████████████████████████████| 112 kB 55.4 MB/s 
     |████████████████████████████████| 149 kB 43.6 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=6488157d47afe9eb218c27a09af59fb69bd63a5329a41aefb47ecbd536ebe2eb
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
import warnings
import gc
import os
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datetime import datetime
from glob import glob
from tqdm import tqdm
from IPython.display import Image

In [ ]:
# 추후에 경고 값들을 출력하지 않도록 'ignore'상태로 만들어 준다.
# 불필요한 경고창이 더 이상 출력 되지 않게 됩니다.
warnings.filterwarnings(action = 'ignore')

### 연도별 데이터를 불러온다.

In [ ]:
train_2017 = []
train_2018 = []
train_2019 = []
train_2020 = []

for i, path in enumerate(sorted(glob('/content/drive/MyDrive/KNOW_data/train/*'))):
  if i == 0 :
    train_2017 = pd.read_csv(path)
    pass
  elif i == 1 :
    train_2018 = pd.read_csv(path)
    pass
  elif i == 2 :
    train_2019 = pd.read_csv(path)
    pass
  else : 
    train_2020 = pd.read_csv(path)
    pass

test_2017 = []
test_2018 = []
test_2019 = []
test_2020 = []

for i, path in enumerate(sorted(glob('/content/drive/MyDrive/KNOW_data/test/*'))):
  if i == 0 :
    test_2017 = pd.read_csv(path)
    pass
  elif i == 1 :
    test_2018 = pd.read_csv(path)
    pass
  elif i == 2 :
    test_2019 = pd.read_csv(path)
    pass
  else : 
    test_2020 = pd.read_csv(path)
    pass

## 2017 데이터 확인

##### 1.1 공백으로 구성된 결측치 값을 np.nan으로 변경

In [ ]:
# 결측치 값 확인을 편하게 하기 위해서 모두 np.nan 값으로 바꿔준다.
# train_2017에 있는 모든 columns들을 불러온다.

for col in train_2017 :
  train_2017[col].replace(' ',np.nan,inplace=True)
  if col != 'knowcode' :
    test_2017[col].replace(' ',np.nan,inplace=True)

KeyError: ignored

##### 1.2 데이터 타입 분포도 확인

In [ ]:
test_2017[test_2017.columns[1:]].dtypes.value_counts()

In [ ]:
test_2017.select_dtypes('float64')

##### 1.3 학습 데이터와 테스트 데이터에서 결측치 값이 포함된 칼럼과 개수 확인

In [ ]:
for index, value, tindex, tvalue in zip(train_2017.isnull().sum().index, train_2017.isnull().sum(), test_2017.isnull().sum().index, test_2017.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

1.4 데이터 결측치 값이 포함된 결측치 값 처리

In [ ]:
# 가번 문항에서 1번을 체크한 사람의 수
train_2017[(train_2017['aq1_2'].isnull()) &  (train_2017['aq1_1'] == 1)].shape

In [ ]:
# 중요하지 않다라고 한 사람들에 대한 결측치 값을 0으로 바꿔준다.
# aq1_1 에서부터 aq41_2 까지의 칼럼들을 뽑아 낸다.

columns = train_2017.columns[1:-1][:82]

# 앞의 결측치 값을 제거하면 뒤의 결측치 값은 제거하면 되기 때문에
# 데이터의 인덱스를 2씩 키운다.
for index in range(0,82,2):
    col = columns[index]
    
    # before가 가번 문항에 해당한다. 
    # after는 나번 문항에 해당한다. 
    before = col
    after = col[:-1] + str((int(col[-1])+1))
    print(before, after)
    
    # 일단 before 자체도 결측치 값이면 안된다.
    # 그리고 after 값이 결측치 값인 것을 확인한다.
    # after 값이 결측치 값이라는 것은 가번 문항에서 1번 을 선택한 것이기 때문에 0으로 할당해 줘야 한다.
    train_2017.loc[(train_2017[after].isnull()) & (train_2017[before].notnull()) & (train_2017[before] == (sorted(train_2017[before].unique()))[0]), after] = 0
    test_2017.loc[(test_2017[after].isnull()) & (test_2017[before].notnull()) & (test_2017[before] == (sorted(test_2017[before].unique()))[0]), after] = 0

In [ ]:
# 결측치 확인
train_nan_columns = []
test_nan_columns = []

for index, value, tindex, tvalue in zip(train_2017.isnull().sum().index, train_2017.isnull().sum(), test_2017.isnull().sum().index, test_2017.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")
    if value != 0 :
        train_nan_columns.append(index)
    if tvalue != 0:
        test_nan_columns.append(tindex)

##### 1.6 aq11_2에 해당하는 결측치 처리 방법


In [ ]:
total_2017 = pd.concat([train_2017[(train_2017['aq11_2'].notnull()) & (train_2017['aq11_1'] == 3)],test_2017[(test_2017['aq11_2'].notnull()) & (test_2017['aq11_1'] == 3)]], axis = 0)

In [ ]:
print(total_2017['aq11_2'].astype('int').describe())

total_2017['aq11_2'].astype('int').plot.hist(bins = 7, title = "aq11_2")

In [ ]:
# 중앙 값으로 결측치 값을 처리해 준다.
# 'train_2017.loc[train_2017['aq11_2'].isnull()] = 4'와 동일한 구문이다.
train_2017.loc[train_2017['aq11_2'].isnull()] = total_2017['aq11_2'].astype('int').quantile(q = 0.5)

# 'test_2017.loc[test_2017['aq11_2'].isnull()] = 4'와 동일한 구문이다.
test_2017.loc[test_2017['aq11_2'].isnull()] = total_2017['aq11_2'].astype('int').quantile(q = 0.5)

##### 1.7 aq14_2에 해당하는 결측치 처리 방법

In [ ]:
train_2017[(train_2017['aq14_2'].isnull())]['aq14_1']

In [ ]:
test_2017[(test_2017['aq14_2'].isnull())]['aq14_1']

In [ ]:
# 가번 문항에서 3번을 선택한 사람의 중앙 값을 알기 위해서 전체 데이터로 합친다.
total_2017 = pd.concat([train_2017[(train_2017['aq14_2'].notnull()) & (train_2017['aq14_1'] == 3)], test_2017[(test_2017['aq14_2'].notnull()) & (test_2017['aq14_1'] == 3)]], axis = 0)
total_2017.shape

In [ ]:
print(total_2017['aq14_2'].astype('int').describe())

total_2017['aq14_2'].astype('int').plot.hist(bins = 7, title = "aq14_2")

In [ ]:
# 가번 문항에서 3번을 선택한 사람들을 나번 문항의 중앙 값을 통해서 처리해준다.

train_2017.loc[(train_2017['aq14_2'].isnull()) & (train_2017['aq14_1'] == 3)] = total_2017['aq14_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq14_2'].isnull()) & (test_2017['aq14_1'] == 3)] = total_2017['aq14_2'].astype('int').quantile(q = 0.5)

In [ ]:
# 가번 문항에서 4번을 선택한 사람의 중앙 값을 알기 위해서 전체 데이터로 합친다.

total_2017 = pd.concat([train_2017[(train_2017['aq14_2'].notnull()) & (train_2017['aq14_1'] == 4)], test_2017[(test_2017['aq14_2'].notnull()) & (test_2017['aq14_1'] == 4)]], axis = 0)
total_2017.shape

In [ ]:
print(total_2017['aq14_2'].astype('int').describe())
total_2017['aq14_2'].astype('int').plot.hist(bins = 7, title = 'aq14_2')

In [ ]:
# 가번 문항에서 4번을 선택한 사람들을 나번 문항의 중앙 값을 통해서 처리해준다.

train_2017.loc[(train_2017['aq14_2'].isnull()) & (train_2017['aq14_1'] == 4)] = total_2017['aq14_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq14_2'].isnull()) & (test_2017['aq14_1'] == 4)] = total_2017['aq14_2'].astype('int').quantile(q = 0.5)

##### 1.8 aq27_2에 해당하는 결측치 처리 방법

In [ ]:
# 가번 문항에서 3번을 선택한 사람을 우선적으로 처리한다.

total_2017 = pd.concat([train_2017[(train_2017['aq27_2'].notnull()) & (train_2017['aq27_1'] == 3)], test_2017[(test_2017['aq27_2'].notnull()) & (test_2017['aq27_1'] == 3)]], axis = 0)

print(total_2017['aq27_2'].astype('int').describe())

total_2017['aq27_2'].astype('int').plot.hist(bins = 7, title = "aq27_2")

train_2017.loc[(train_2017['aq27_2'].isnull()) & (train_2017['aq27_1'] == 3)] = total_2017['aq27_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq27_2'].isnull()) & (test_2017['aq27_1'] == 3)] = total_2017['aq27_2'].astype('int').quantile(q = 0.5)

In [ ]:
# 가번 문항에서 4번을 선택한 사람을 처리한다.

total_2017 = pd.concat([train_2017[(train_2017['aq27_2'].notnull()) & (train_2017['aq27_1'] == 4)], test_2017[(test_2017['aq27_2'].notnull()) & (test_2017['aq27_1'] == 4)]], axis = 0)

print(total_2017['aq27_2'].astype('int').describe())

total_2017['aq27_2'].astype('int').plot.hist(bins = 7, title = "aq27_2")

train_2017.loc[(train_2017['aq27_2'].isnull()) & (train_2017['aq27_1'] == 4)] = total_2017['aq27_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq27_2'].isnull()) & (test_2017['aq27_1'] == 4)] = total_2017['aq27_2'].astype('int').quantile(q = 0.5)

##### 1.9 aq32_2에 해당하는 결측치 처리 방법

In [ ]:
train_2017[(train_2017['aq32_2'].isnull())]['aq32_1']

In [ ]:
test_2017.loc[(test_2017['aq32_1'] == 33), 'aq32_1'] = 3
test_2017[(test_2017['aq32_2'].isnull())]['aq32_1']

In [ ]:
# 가번 문항을 3으로 선택한 사람들에 대해 결측치 값을 처리한다.

total_2017 = pd.concat([train_2017[(train_2017['aq32_2'].notnull()) & (train_2017['aq32_1'] == 3)], test_2017[(test_2017['aq32_2'].notnull()) & (test_2017['aq32_1'] == 3)]], axis = 0)

print(total_2017['aq32_2'].astype('int').describe())

total_2017['aq32_2'].astype('int').plot.hist(bins = 7, title = "aq32_1")

train_2017.loc[(train_2017['aq32_2'].isnull()) & (train_2017['aq32_1'] == 3)] = total_2017['aq32_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq32_2'].isnull()) & (test_2017['aq32_1'] == 3)] = total_2017['aq32_2'].astype('int').quantile(q = 0.5)

##### aq41_2에 해당하는 결측치 처리 방법

In [ ]:
# 가번 문항을 3으로 선택한 사람들에 대한 결측치 값을 처리한다. 

total_2017 = pd.concat([train_2017[(train_2017['aq41_2'].notnull()) & (train_2017['aq41_1'] == 3)], test_2017[(test_2017['aq41_2'].notnull()) & (test_2017['aq41_1'] == 3)]], axis = 0)

print(total_2017['aq41_2'].astype('int').describe())

total_2017['aq41_2'].astype('int').plot.hist(bins = 7, title = "aq41_2")

train_2017.loc[(train_2017['aq41_2'].isnull()) & (train_2017['aq41_1'] == 3)] = total_2017['aq41_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq41_2'].isnull()) & (test_2017['aq41_1'] == 3)] = total_2017['aq41_2'].astype('int').quantile(q = 0.5)

In [ ]:
for index, value, tindex, tvalue in zip(train_2017.isnull().sum().index, train_2017.isnull().sum(), test_2017.isnull().sum().index, test_2017.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

##### 1.11 bq4_1a ~ bq4_1c에 해당하는 결측치 처리 방법

In [ ]:
print(train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1a'].isnull())].shape)
print(test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1a'].isnull())].shape)

In [ ]:
train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1a'].isnull()), 'bq4_1a'] = '없음'

In [ ]:
# 정상적으로 결측치 값이 제거되었다.
display(train_2017[(train_2017['bq4_1a'].isnull())])
display(test_2017[(test_2017['bq4_1a'].isnull())])

In [ ]:
# 4번 문항에서 2번을 선택한 사람이라면 나머지 요구되는 자격증 모두 '없음' 이다.

train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

bq4_1a에 동일한 자격증을 갖고 있지만 bq4_1b에는 비어 있는 경우도 있다.  
"귀하의 업무를 수행하는데 요구되는 자격증" 이기 때문에 같은 자격증을 갖고 있다는 것은 비슷한 업무를 할 확률이 높다.  
따라서 bq4_1b에는 동일한 자격증을 기입해 준다.

In [ ]:
for index in test_2017.loc[(test_2017['bq4_1b'].isnull())].index:
    value = test_2017.iloc[index]['bq4_1a']
#     col_index = np.where(test_2017.columns == 'bq4_1a')[0][0]
    
    if len(test_2017[(test_2017['bq4_1a'] == value) & (test_2017['bq4_1b'].notnull()) & (test_2017['bq4_1b'] != '없음') & (test_2017['bq4_1b'] != '없다')]) > 0:
        test_2017.loc[index, 'bq4_1b'] = test_2017[(test_2017['bq4_1a'] == value) & (test_2017['bq4_1b'].notnull()) & (test_2017['bq4_1b'] != '없음') & (test_2017['bq4_1b'] != '없다')]['bq4_1b'].value_counts().index[0]

In [ ]:
# 위에 해당하는 사항이 없다면 '없음' 값으로 초기화 한다.
train_2017.loc[(train_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2017.loc[(test_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'

In [ ]:
train_2017.loc[(train_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2017.loc[(test_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [ ]:
for index, value, tindex, tvalue in zip(train_2017.isnull().sum().index, train_2017.isnull().sum(), test_2017.isnull().sum().index, test_2017.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

In [ ]:
for col in test_2017 : 
  train_2017[col].replace('없다','없음',inplace = True)
  train_2017[col].replace('없다','없음',inplace = True)

##### 1.12 bq5_1 ~ bq5_2 해당하는 결측치 처리 방법

In [ ]:
print("학습 데이터 결측치 : ", train_2017[(train_2017['bq5'] == 2) & (train_2017['bq5_1'].isnull())].shape)
print("테스트 데이터 결측치 : ", test_2017[(test_2017['bq5'] == 2) & (test_2017['bq5_1'].isnull())].shape)

In [ ]:
print("학습 데이터 결측치 : ", train_2017[(train_2017['bq5'] == 2) & (train_2017['bq5_2'].isnull())].shape)
print("테스트 데이터 결측치 : ", test_2017[(test_2017['bq5'] == 2) & (test_2017['bq5_2'].isnull())].shape)

In [ ]:
# 5번 문항에서 2번을 선택한 사람들에 대해서 5-1에 대한 값을 0으로 처리함
train_2017.loc[(train_2017['bq5'] == 2) & (train_2017['bq5_1'].isnull()),'bq5_1'] = 0
test_2017.loc[(test_2017['bq5'] == 2) & (test_2017['bq5_1'].isnull()),'bq5_1'] = 0

In [ ]:
# 5번 문항에서 2번을 선택한 사람들에 대해서 5-2에 대한 값을 '없음'으로 처리함
train_2017.loc[(train_2017['bq5'] == 2) & (train_2017['bq5_2'].isnull()),'bq5_2'] = '없음'
test_2017.loc[(test_2017['bq5'] == 2) & (test_2017['bq5_2'].isnull()),'bq5_2'] = '없음'

In [ ]:
# 학습 데이터와 테스트 데이터에서 한 개씩 결측치 값이 남는다.
display(train_2017[train_2017['bq5_2'].isnull()])
display(test_2017[test_2017['bq5_2'].isnull()])

In [ ]:
# train과 test 모두 필요한 자격증 값이 '없음'이기 때문에 단순히 유추하기 어렵다.
# 따라서, '없음'으로 전처리 해준다.
train_2017.loc[train_2017['bq5_2'].isnull(), 'bq5_2'] = '없음'
test_2017.loc[test_2017['bq5_2'].isnull(), 'bq5_2'] = '없음'

##### 1.13 bq12 해당하는 결측치 처리 방법

In [ ]:
# bq12_2 컬럼에 대한 결측치를 제거한다.

for index in train_2017.loc[(train_2017['bq12_2'].isnull())].index:
    # 학력 값을 저장함
    LevelOfEducation = train_2017.iloc[index]['bq7']

    # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
    train_2017.loc[index, 'bq12_2'] = train_2017[(train_2017['bq7'] == LevelOfEducation) & (train_2017['bq12_2'].notnull())]['bq12_2'].astype('int').quantile(q=0.5).astype('int').astype('str')

In [ ]:
train_2017[(train_2017['bq7'] == LevelOfEducation) & (train_2017['bq12_2'].notnull())]['bq12_2'].astype('int').quantile(q=0.5).astype('int')

In [ ]:
# bq12_2 컬럼에 대한 결측치를 제거한다.

for index in test_2017.loc[(test_2017['bq12_2'].isnull())].index:
    # 학력 값을 저장함
    LevelOfEducation = test_2017.iloc[index]['bq7']

    # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
    test_2017.loc[index, 'bq12_2'] = test_2017[(test_2017['bq7'] == LevelOfEducation) & (test_2017['bq12_2'].notnull())]['bq12_2'].astype('int').quantile(q=0.5).astype('int').astype('str')

In [ ]:
# bq12_3 컬럼에 대한 결측치를 제거한다.

for index in train_2017.loc[(train_2017['bq12_3'].isnull())].index:
    # 학력 값을 저장함
    LevelOfEducation = train_2017.iloc[index]['bq7']

    # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
    train_2017.loc[index, 'bq12_3'] = train_2017[(train_2017['bq7'] == LevelOfEducation) & (train_2017['bq12_3'].notnull())]['bq12_3'].astype('int').quantile(q=0.5).astype('int').astype('str')
    
for index in test_2017.loc[(test_2017['bq12_3'].isnull())].index:
    # 학력 값을 저장함
    LevelOfEducation = test_2017.iloc[index]['bq7']

    # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
    test_2017.loc[index, 'bq12_3'] = test_2017[(test_2017['bq7'] == LevelOfEducation) & (test_2017['bq12_3'].notnull())]['bq12_3'].astype('int').quantile(q=0.5).astype('int').astype('str')    

In [ ]:
# bq12_4 컬럼에 대한 결측치를 제거한다.

for index in train_2017.loc[(train_2017['bq12_4'].isnull())].index:
    # 학력 값을 저장함
    LevelOfEducation = train_2017.iloc[index]['bq7']

    # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
    train_2017.loc[index, 'bq12_4'] = train_2017[(train_2017['bq7'] == LevelOfEducation) & (train_2017['bq12_4'].notnull())]['bq12_4'].astype('int').quantile(q=0.5).astype('int').astype('str')

for index in test_2017.loc[(test_2017['bq12_4'].isnull())].index:
    # 학력 값을 저장함
    LevelOfEducation = test_2017.iloc[index]['bq7']

    # 학력 값에 대한 중앙값을 통해서 결측치를 제거한다.
    test_2017.loc[index, 'bq12_4'] = test_2017[(train_2017['bq7'] == LevelOfEducation) & (test_2017['bq12_4'].notnull())]['bq12_4'].astype('int').quantile(q=0.5).astype('int').astype('str')

##### bq19_1 해당하는 결측치 처리 방법

In [ ]:
train_2017.loc[train_2017['bq19_1'].isnull()] = '없음'
test_2017.loc[test_2017['bq19_1'].isnull()] = '없음'

In [ ]:
# 직전에 직업이 었는 사람인 것을 확인 했다.
# 따라서 별도의 직업을 달리 부르는 명칭이 존재하지 않는다.
print(test_2017[test_2017['bq30'].isnull()]['bq32'])
test_2017.loc[test_2017['bq30'].isnull()] = '없음'

##### 1.15 bq31 해당하는 결측치 처리 방법

In [ ]:
for index, value, tindex, tvalue in zip(train_2017.isnull().sum().index, train_2017.isnull().sum(), test_2017.isnull().sum().index, test_2017.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

In [ ]:
for index in train_2017[(train_2017['bq31'].isnull())].index:
    # 산업 유형
    industry_type = train_2017.iloc[index]['bq1']
    # 유사직업명
    a_similar_job = train_2017.iloc[index]['bq30']
    # 직전직업
    before_job = train_2017.iloc[index]['bq32']
    # 전직 가능직업
    can_change_job = train_2017.iloc[index]['bq33']
    

    if len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업 + 전직 가능 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
        #  유사 직업명 + 직전 직업 + 전직 가능 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job)]):
        # 산업 유형 + 유사 직업명
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]):
        # 산업 유형 + 직전 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
        # 직전 직업 + 전직 가능직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job)]):
        # 직전 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job)].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull()) & (train_2017['bq33'] == can_change_job)]):
        # 전직 가능직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull()) & (train_2017['bq33'] == can_change_job)].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job)]):
        # 유사 직업명
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job)].value_counts().index[0]
    else:
        # 이정도면 없음으로 할게요...
        train_2017.loc[index, 'bq31'] = '없음'

In [ ]:
for index in test_2017[(test_2017['bq31'].isnull())].index:
    # 산업 유형
    industry_type = test_2017.iloc[index]['bq1']
    # 유사직업명
    a_similar_job = test_2017.iloc[index]['bq30']
    # 직전직업
    before_job = test_2017.iloc[index]['bq32']
    # 전직 가능직업
    can_change_job = test_2017.iloc[index]['bq33']

    if len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job) & (
                              test_2017['bq33'] == can_change_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업 + 전직 가능 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job) & (
                        test_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)][
            'bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job) & (
            test_2017['bq32'] == before_job) & (test_2017['bq33'] == can_change_job)]):
        #  유사 직업명 + 직전 직업 + 전직 가능 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job) & (
                        test_2017['bq32'] == before_job) & (test_2017['bq33'] == can_change_job)][
            'bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job)]):
        # 산업 유형 + 유사 직업명
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job)]['bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)]):
        # 산업 유형 + 직전 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)].value_counts().index[
            0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job) & (
            test_2017['bq33'] == can_change_job)]):
        # 직전 직업 + 전직 가능직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job) & (
                        test_2017['bq33'] == can_change_job)].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job)]):
        # 직전 직업
        test_2017.loc[index, 'bq31'] = \
        test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job)].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq33'] == can_change_job)]):
        # 전직 가능직업
        test_2017.loc[index, 'bq31'] = \
        test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq33'] == can_change_job)].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job)]):
        # 유사 직업명
        test_2017.loc[index, 'bq31'] = \
        test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job)].value_counts().index[0]
    else:
        # 이정도면 없음으로 할게요...
        test_2017.loc[index, 'bq31'] = '없음'

##### 1.16 bq32 해당하는 결측치 처리 방법

In [ ]:
train_2017.loc[train_2017['bq32'].isnull(), 'bq32'] = '모름' 
test_2017.loc[test_2017['bq32'].isnull(), 'bq32'] = '모름' 

사실은 bq31과 같은 사용 가능한 프로그램 같은 경우는 프로그램의 종류 별로 컬럼을 새로 만드는 것이 좋다고 생각합니다. 이를 통해서 bq33 결측치 값을 좀 더 세밀하게 처리할 수 있습니다 Ex) 엑셀, 워드를 다룰 수 있다면 엑셀과 워드에 해당하는 컬럼을 따로 만드는 것이 좋다(원-핫 인코딩 과 같은)

일부 컬럼에서 의미가 같은 단어를 다른 방식으로 표현하는 부분들이 많다. Ex) 컴퓨터와 PC를 혼용해서 사용하는 컬럼들이 보인다.

이는 모델링 작업시 모두 동일한 형태로 변환 해주는 것이 좋다고 생각합니다.


##### 1.17 bq34 해당하는 결측치 처리 방법

In [ ]:
train_2017.loc[(train_2017['bq34'].isnull()), 'bq34'] = '모름'
test_2017.loc[(test_2017['bq34'].isnull()), 'bq34'] = '모름'

In [ ]:
print(f"train : {train_2017[train_2017['bq38_1'].isnull()]['bq38'].shape[0]}")
print(f"test : {test_2017[test_2017['bq38_1'].isnull()]['bq38'].shape[0]}")

In [ ]:
train_2017.loc[train_2017['bq38_1'].isnull()] = '없음'
test_2017.loc[test_2017['bq38_1'].isnull()] = '없음'

##### 1.19 bq40 해당하는 결측치 처리 방법

In [ ]:
print(train_2017[(train_2017['bq40'].isnull())]['bq39_2'].value_counts())
print(test_2017[(test_2017['bq40'].isnull())]['bq39_2'].value_counts())

In [ ]:
train_2017.loc[(train_2017['bq39_1'] == 2) & (train_2017['bq40'].isnull())] = '3'
test_2017.loc[(test_2017['bq39_1'] == 2) & (test_2017['bq40'].isnull())] = '3'

In [ ]:
train_2017 = train_2017.drop(columns = ['bq41_1', 'bq41_2', 'bq41_3'])
test_2017 = test_2017.drop(columns = ['bq41_1', 'bq41_2', 'bq41_3'])

In [ ]:
for index, value, tindex, tvalue in zip(train_2017.isnull().sum().index, train_2017.isnull().sum(), test_2017.isnull().sum().index, test_2017.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

In [ ]:
train_2017.to_csv("220114_know2017_train.csv")
test_2017.to_csv("220114_know2017_test.csv")